# Exemple de génération de trace GPX à partir des points de la trace RTK exportée en format GeoJSON depuis l'application SW MAPS

## Données d'entrée : fichier geojson des points de la trace RTK collectés lors de la sortie de terrain - Pole d'activités d'Aix en Provence :

### 1) Installation de la bibliothèque python gpxpy pour la création de la trace GPX :

In [18]:
!pip install gpxpy

   ---------------------------------------- 0.0/42.6 kB ? eta -:--:--
   ---------------------------------------- 42.6/42.6 kB 1.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2) Installation de la bibliothèque python geojson pour le chargement du fichier geojson d'entrée :

In [20]:
!pip install geojson


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 3) Extraction des points RTK avec leurs horodatages en liste et les convertir en DataFrame :

In [46]:
import geojson
import pandas as pd

# Charger le fichier GeoJSON
with open(r"C:\Users\alaeddine.jerad\Desktop\stage CIM\sig et data\Sortie gopro + RTK 19-06 PAAP\Trace PAAP points.geojson", 'r') as f:
    data = geojson.load(f)

# Extraire les points et leurs horodatages
points = []
for feature in data['features']:
    if feature['geometry']['type'] == 'Point':
        coords = feature['geometry']['coordinates']
        timestamp = feature['properties'].get('time')
        points.append({
            'latitude': coords[1],
            'longitude': coords[0],
            'elevation': coords[2] if len(coords) > 2 else None,
            'time': timestamp
        })

# Convertir en DataFrame
df = pd.DataFrame(points)
df

,latitude,longitude,elevation,time
0,43.491988,5.373602,165.039,19-06-2024 15:33:37.200 GMT+02:00
1,43.491992,5.373568,164.817,19-06-2024 15:33:42.800 GMT+02:00
2,43.491977,5.373481,164.816,19-06-2024 15:33:45.200 GMT+02:00
3,43.491904,5.373417,165.000,19-06-2024 15:33:47.600 GMT+02:00
4,43.491807,5.373362,164.871,19-06-2024 15:33:50.000 GMT+02:00
...,...,...,...,...
223,43.491668,5.373427,165.475,19-06-2024 15:43:53.200 GMT+02:00
224,43.491713,5.373537,165.547,19-06-2024 15:43:55.600 GMT+02:00
225,43.491728,5.373660,165.633,19-06-2024 15:43:58.000 GMT+02:00
226,43.491718,5.373787,165.701,19-06-2024 15:44:00.400 GMT+02:00


### 4) Installation de la bibliothèque python dateutil pour le formatage de la date à partir d'une chaine de caractères :

In [40]:
pip install python-dateutil

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 5) Création de la trace GPX en ajoutant les points horodatées au segment de la trace :

In [47]:
import gpxpy
import gpxpy.gpx
from dateutil import parser

# Créer un objet GPX
gpx = gpxpy.gpx.GPX()

# Créer une nouvelle trace
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)

# Ajouter un segment à la trace
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)

# Ajouter les points au segment
for idx, row in df.iterrows():
# Convertir l'horodatage en format datetime
    time = parser.parse(row['time']) if row['time'] else None
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(
        latitude=row['latitude'],
        longitude=row['longitude'],
        elevation=row['elevation'],
        time=time
    ))

### 6) Exportation de la trace GPX :

In [48]:
# Écrire le fichier GPX
with open(r"C:\Users\alaeddine.jerad\Desktop\stage CIM\sig et data\Sortie gopro + RTK 19-06 PAAP\Trace PAAP points.gpx", 'w') as f:
    f.write(gpx.to_xml())